#### Companion reading if you want to understand how it works:  

[Creating a (mostly) Autonomous HR Assistant with ChatGPT and LangChain’s Agents and Tools](https://medium.com/@stephen.bonifacio/creating-a-mostly-autonomous-hr-assistant-with-chatgpt-and-langchains-agents-and-tools-1cdda0aa70ef)



In [ ]:
# load core modules
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
# load agents and tools modules
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from io import StringIO
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

# initialize pinecone client and connect to pinecone index
pinecone.init(
        api_key="<your pinecone api key>",  
        environment="your pinecone environment"  
) 

index_name = 'tk-policy'
index = pinecone.Index(index_name) # connect to pinecone index


# initialize embeddings object; for use with user query/input
embed = OpenAIEmbeddings(
                deployment="<your deployment name>",
                model="text-embedding-ada-002",
                openai_api_key='<your azure openai api key>',
                openai_api_base="<your api base>",
                openai_api_type="azure",
            )

# initialize langchain vectorstore(pinecone) object
text_field = 'text' # key of dict that stores the text metadata in the index
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

# initialize LLM object
llm = AzureChatOpenAI(    
    deployment_name="<your deployment name>", 
    model_name="gpt-35-turbo", 
    openai_api_key='<your openai api key>',
    openai_api_version = '2023-03-15-preview', 
    openai_api_base='<your api base>',
    openai_api_type='azure'
    )

# initialize vectorstore retriever object
timekeeping_policy = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

# create employee data tool 
client = DataLakeServiceClient( # authenticate to azure datalake
                               account_url="<you azure account storage url>",
                               credential="<your azure storage account keys>"
                              )
# azure data lake boilerplate to load from file system.  
file = client.get_file_system_client("<your azure storage account name>") \
             .get_file_client("employee_data/employee_data.csv") \
             .download_file() \
             .readall() \
             .decode('utf-8') 

csv_file = StringIO(file) 
df = pd.read_csv(csv_file) # load employee_data.csv as dataframe
python = PythonAstREPLTool(locals={"df": df}) # set access of python_repl tool to the dataframe

# create calculator tool
calculator = LLMMathChain.from_llm(llm=llm, verbose=True)

# create variables for f strings embedded in the prompts
user = 'Alexander Verdad' # set user
df_columns = df.columns.to_list() # print column names of df

# prep the (tk policy) vectordb retriever, the python_repl(with df access) and langchain calculator as tools for the agent
tools = [
    Tool(
        name = "Timekeeping Policies",
        func=timekeeping_policy.run,
        description="""
        Useful for when you need to answer questions about employee timekeeping policies.

        <user>: What is the policy on unused vacation leave?
        <assistant>: I need to check the timekeeping policies to answer this question.
        <assistant>: Action: Timekeeping Policies
        <assistant>: Action Input: Vacation Leave Policy - Unused Leave
        ...
        """
    ),
    Tool(
        name = "Employee Data",
        func=python.run,
        description = f"""
        Useful for when you need to answer questions about employee data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: {df_columns}
        
        <user>: How many Sick Leave do I have left?
        <assistant>: df[df['name'] == '{user}']['sick_leave']
        <assistant>: You have n sick leaves left.              
        """
    ),
    Tool(
        name = "Calculator",
        func=calculator.run,
        description = f"""
        Useful when you need to do math operations or arithmetic.
        """
    )
]

# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
agent_kwargs = {'prefix': f'You are friendly HR assistant. You are tasked to assist the current user: {user} on questions related to HR. You have access to the following tools:'}


# initialize the LLM agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True, 
                         agent_kwargs=agent_kwargs
                         )
# define q and a function for frontend
def get_response(user_input):
    response = agent.run(user_input)
    return response

In [21]:
print(agent.agent.llm_chain.prompt.template)

You are friendly HR assistant. You are tasked to assist the current user: Alexander Verdad on questions related to HR. You have access to the following tools:

Timekeeping Policies: 
        Useful for when you need to answer questions about employee timekeeping policies.
        
Employee Data: 
        Useful for when you need to answer questions about employee data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: ['employee_id', 'name', 'position', 'organizational_unit', 'rank', 'hire_date', 'regularization_date', 'vacation_leave', 'sick_leave', 'basic_pay_in_php', 'employment_status', 'supervisor']
        
        <user>: How many Sick Leave do I have left?
        <assistant>: df[df['name'] == 'Alexander Verdad']['sick_leave']
        <assistant>: You have n sick leaves left.              
        
Calculator: 
        Useful when you need to do math operations or arithmetic.
 

In [22]:
get_response('What is my name and employee id?')



> Entering new AgentExecutor chain...
I should use the Employee Data tool to find the information.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad'][['employee_id', 'name']]
Observation:    employee_id              name
1      1005676  Alexander Verdad
Thought:I have found the answer.
Final Answer: Your name is Alexander Verdad and your employee id is 1005676.

> Finished chain.


'Your name is Alexander Verdad and your employee id is 1005676.'

In [22]:
get_response('Are employees on probation allowed to have vacation leaves?')



> Entering new AgentExecutor chain...
I need to check the company's timekeeping policies to see if probationary employees are allowed to take vacation leaves.
Action: Timekeeping Policies
Action Input: Search for "vacation leave" in the timekeeping policies document
Observation: I'm sorry, but I cannot perform that task as the context provided only includes the HR Policy Manual - Leave Policy and the HR Policy Manual - Attendance Policy. There is no mention of a timekeeping policies document.
Thought:I need to check the HR Policy Manual - Leave Policy to see if probationary employees are allowed to take vacation leaves.
Action: Timekeeping Policies
Action Input: Search for "probationary" in the HR Policy Manual - Leave Policy
Observation: In the HR Policy Manual - Leave Policy, under the section "III. LEAVE POLICY," in subsection A. Vacation Leave, point 5, it is mentioned that "During Probation: Employees on probation are not eligible for Vacation Leave." In subsection C. Service In

'No, probationary employees are not allowed to have vacation leaves according to the HR Policy Manual - Leave Policy.'

In [23]:
get_response("How many vacation leaves do I have left and what is the policy on unused VLs?")



> Entering new AgentExecutor chain...
I need to check the employee data to get the number of vacation leaves left and the timekeeping policies to know the policy on unused VLs.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad']['vacation_leave']
Observation: 1    45
Name: vacation_leave, dtype: int64
Thought:Now I need to check the timekeeping policies to know the policy on unused VLs.
Action: Timekeeping Policies
Action Input: Check the company's employee handbook or policy manual for information on unused VLs.
Observation: According to the HR Policy Manual - Leave Policy, unused Vacation Leave can be carried over to the next year. However, the total accumulated leave should not exceed 30 days. Any excess leave will be forfeited. Unused Vacation Leave can also be encashed at the end of the year at the basic salary rate. Encashment rate is basic salary divided by 30 days multiplied by unused leaves to be encashed.
Thought:I now know the policy on unused VLs and 

'You have 45 vacation leaves left and unused VLs can be carried over to the next year but should not exceed 30 days. Any excess leave will be forfeited. Unused VLs can also be encashed at the end of the year at the basic salary rate. Encashment rate is basic salary divided by 30 days multiplied by unused leaves to be encashed.'

In [24]:
get_response("How much will I be paid if I encash my unused VLs?")



> Entering new AgentExecutor chain...
I need to check the timekeeping policies and the employee data to calculate the amount.
Action: Timekeeping Policies
Action Input: Check the policy on VL encashment
Observation: According to the HR Policy Manual - Leave Policy, Vacation Leave can be encashed at the end of the year at the basic salary rate. The encashment rate is basic salary divided by 30 days multiplied by unused leaves to be encashed.
Thought:Now I need to get the basic salary rate and the number of unused VLs from the employee data.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad'][['basic_pay_in_php', 'vacation_leave']]
Observation:    basic_pay_in_php  vacation_leave
1             15000              45
Thought:Now I can calculate the amount.
Action: Calculator
Action Input: (15000/30)*45

> Entering new LLMMathChain chain...
(15000/30)*45```text
(15000/30)*45
```
...numexpr.evaluate("(15000/30)*45")...

Answer: 22500.0
> Finished chain.

Observation: A

'You will be paid Php 22,500.00 if you encash your unused VLs.'